In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv
/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv
/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv


In [2]:
train_df=pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv")

In [3]:
train_df.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [4]:
train_df.drop("id" , axis = 1 ,inplace=True)

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   article     287113 non-null  object
 1   highlights  287113 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB


In [6]:
train_df.iloc[0]

article       By . Associated Press . PUBLISHED: . 14:11 EST...
highlights    Bishop John Folda, of North Dakota, is taking ...
Name: 0, dtype: object

In [7]:
train_df.iloc[0]['article']

"By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through contaminated food while attending a conference for newly ordained 

In [8]:
script = train_df.iloc[0]['article']

In [9]:
train_df.iloc[0]['highlights']

'Bishop John Folda, of North Dakota, is taking time off after being diagnosed .\nHe contracted the infection through contaminated food in Italy .\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed .'

In [10]:
summary = train_df.iloc[0]['highlights']

In [11]:
summary

'Bishop John Folda, of North Dakota, is taking time off after being diagnosed .\nHe contracted the infection through contaminated food in Italy .\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed .'

In [12]:
from huggingface_hub import notebook_login
notebook_login()

In [13]:
train_df.columns

Index(['article', 'highlights'], dtype='object')

In [14]:
!pip install --upgrade pip
!pip install torch==2.5.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install "unsloth[cu124-torch250] @ git+https://github.com/unslothai/unsloth.git"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 MB 40.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 45.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 67.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 112.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 131.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 113.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 98.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 1

In [15]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-12 15:39:09.164276: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747064349.424440      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747064349.494573      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [16]:
def find_target_modules(model, target_class_name="Linear4bit", return_full_names=False):
    """
    Find all unique module names in the model that match a given class name substring.

    Args:
        model (torch.nn.Module): The model to inspect.
        target_class_name (str): Substring to look for in module types (e.g., "Linear4bit").
        return_full_names (bool): If True, return full module names; otherwise, just the last part.

    Returns:
        List[str]: Unique module name parts where the target class was found.
    """
    unique_layers = set()

    for name, module in model.named_modules():
        if target_class_name in type(module).__name__:
            layer_name = name if return_full_names else name.split('.')[-1]
            unique_layers.add(layer_name)

    return sorted(unique_layers)


In [17]:
res = find_target_modules(model, target_class_name="Linear4bit", return_full_names=False)
print(res)

['down_proj', 'gate_proj', 'k_proj', 'o_proj', 'q_proj', 'up_proj', 'v_proj']


In [18]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = res,
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None # And LoftQ
)

Unsloth 2025.5.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [19]:
subset_df = train_df.sample(n=5000, random_state=42).reset_index(drop=True)

In [20]:
subset_df

,article,highlights
0,By . Mia De Graaf . Britons flocked to beaches...,People enjoyed temperatures of 17C at Brighton...
1,A couple who weighed a combined 32st were sham...,Couple started piling on pounds after the birt...
2,Video footage shows the heart stopping moment ...,A 17-year-old boy suffering lacerations to his...
3,"Istanbul, Turkey (CNN) -- About 250 people rac...",Syrians citizens hightail it to Turkey .\nMost...
4,By . Daily Mail Reporter . PUBLISHED: . 12:53 ...,The Xue Long had provided the helicopter that ...
...,...,...
4995,"By . Matt Chorley, Mailonline Political Editor...",Major General Jonathan Shaw accuses ministers ...
4996,"ST. POELTEN, Austria (CNN) -- A verdict in th...","Friztl pleads guilty to imprisonment, incest d..."
4997,"By . Hugo Gye . PUBLISHED: . 07:49 EST, 22 Jan...","Ex-footballer, 43, has repeatedly been targete..."
4998,"(CNN) -- It's no Super Bowl. Heck, it's no Mon...",ESPN moves English club soccer game to flagshi...


In [21]:
from datasets import Dataset

# 1. Define the Alpaca-style prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Summarize the following text:

### Input:
{}

### Response:
{}"""

# 2. Get the EOS token from your tokenizer
EOS_TOKEN = tokenizer.eos_token

# 3. Define the formatting function
def formatting_prompts_func(examples):
    inputs = examples["article"]     # raw input text
    outputs = examples["highlights"] # summary (fixed typo)
    texts = [alpaca_prompt.format(inp, out) + EOS_TOKEN for inp, out in zip(inputs, outputs)]
    return { "text": texts }

# 4. Convert your DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(subset_df)

# 5. Apply the formatting function
dataset = dataset.map(formatting_prompts_func, batched=True)


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [22]:
dataset

Dataset({
    features: ['article', 'highlights', 'text'],
    num_rows: 5000
})

In [23]:
from transformers import AutoTokenizer

# Load tokenizer for Unsloth's Meta-Llama-3.1-8B model
tokenizer = AutoTokenizer.from_pretrained("unsloth/Meta-Llama-3.1-8B", use_fast=True)


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [24]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True,  # Speeds up training for short text inputs
    args = TrainingArguments(
        per_device_train_batch_size = 4,       # Increased for speed
        gradient_accumulation_steps = 1,       # Simpler, higher throughput
        num_train_epochs = 1,                  # Faster experimentation
        warmup_steps = 10,
        learning_rate = 5e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none"
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/5000 [00:00<?, ? examples/s]

Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!


In [25]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
7.137 GB of memory reserved.


In [26]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1 | Total steps = 625
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 20,971,520/8,000,000,000 (0.26% trained)


Step,Training Loss
5,2.253500
10,2.283700
15,2.129600
20,2.131800
25,2.108400
30,2.086900
35,1.969200
40,1.940800
45,2.029500
50,2.019900


Unsloth: Will smartly offload gradients to save VRAM!


In [42]:
script_2 = train_df.iloc[10]['article']

In [43]:
script_2

"By . Ellie Zolfagharifard . Take a look at a map today, and you’re likely to see that North America is larger than Africa, Alaska is larger than Mexico and China is smaller than Greenland. But in reality China is four times bigger than Greenland, Africa is three times bigger than North America and Mexico is larger than Alaska. The distortion is the result of the Mercator projection, the map most commonly seen hanging in classrooms and in text books, which was created in 1596 to help sailors navigate the world. The Mercator projection, the map most commonly seen hanging in classrooms and in text books, was created in 1596 to help sailors navigate the world. The familiar map gives the right shapes of land masses, but at the cost of distorting their sizes in favour of the wealthy lands to the north . You might think that the advent of satellite imagery and tools such as Google maps has improved our view of the world, but this isn’t necessarily the case, according to James Wan writing in 

In [78]:
from unsloth import FastLanguageModel
import torch, re

# Enable fast inference mode
FastLanguageModel.for_inference(model)

# Add special tokens to ensure proper stopping
if tokenizer.eos_token is None:
    tokenizer.eos_token = "</s>"
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Ultra-simplified prompt with explicit end markers
prompt = f"""<|im_start|>system
You are a summarization AI. Generate only a 2-3 sentence summary. Always complete your sentences.
<|im_end|>
<|im_start|>user
Summarize this article in 2-3 complete sentences:

{script_2}
<|im_end|>
<|im_start|>assistant
"""

# Add explicit ending tokens to the tokenizer's vocabulary if not present
end_token = "<|im_end|>"
if end_token not in tokenizer.get_vocab():
    print("End token not in vocabulary, using EOS token instead")
    end_token = tokenizer.eos_token

# Tokenize
inputs = tokenizer(
    [prompt],
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=2048,
).to("cuda")

# Force the model to generate a complete summary
outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    min_new_tokens=20,
    temperature=0.1,  # Very low temperature to reduce randomness
    top_p=0.5,       # More restrictive top_p
    do_sample=False, # Turn off sampling for more deterministic output
    num_beams=None,     # Use beam search for better completion
    early_stopping=True,
    repetition_penalty=1.0,  # Default repetition penalty
    length_penalty=1.0,      # No length penalty
    use_cache=True
)

# Get the generated text
full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
input_text = tokenizer.decode(inputs.input_ids[0], skip_special_tokens=True)
summary = full_output[len(input_text):].strip()

# Final cleanup to ensure we have only the summary
summary = re.sub(r'(?i)(summary:|the summary is:|here\'s a summary:|step-by-step|instructions:|note:|###)', '', summary)
summary = re.sub(r'^\d+\.[\s]*', '', summary, flags=re.MULTILINE)
summary = re.sub(r'^\*[\s]*', '', summary, flags=re.MULTILINE)
summary = re.sub(r'<\|im_end\|>.*', '', summary, flags=re.MULTILINE)




, '', summary, flags=re.DOTALL)  # Remove anything after end token

# Ensure the summary ends with proper punctuation
if summary and not summary[-1] in ['.', '!', '?']:
    # Find the last complete sentence
    last_sentence_end = max(summary.rfind('.'), summary.rfind('!'), summary.rfind('?'))
    if last_sentence_end > 0:
        summary = summary[:last_sentence_end+1]

print("SUMMARY:\n", summary.strip())

# Post-processing function to call if you still get incomplete summaries
def get_complete_sentences(text):
    """Extract only complete sentences from text"""
    sentences = re.findall(r'[^.!?]*[.!?]', text)
    if len(sentences) == 0:
        return text  # Return original if no complete sentences found
    return ' '.join(sentences)

# Uncomment this line if you still get incomplete summaries
# print("CLEANED SUMMARY:\n", get_complete_sentences(summary))

End token not in vocabulary, using EOS token instead
SUMMARY:
 By. Ellie Zolfagharifard. Take a look at a map today, and you’re likely to see that North America is larger than Africa, Alaska is larger than Mexico and China is smaller than Greenland. But in reality China is four times bigger than Greenland, Africa is three times bigger than North America and Mexico is larger than Alaska. The distortion is the result of the Mercator projection, the map most commonly seen hanging in classrooms and in text books, which was created in 1596 to help sailors navigate the world. The Mercator projection, the map most commonly seen hanging in classrooms and in text books, was created in 1596 to help sailors navigate the world. The familiar map gives the right shapes of land masses, but at the cost of distorting their sizes in favour of the wealthy lands to the north.


In [76]:
high_2 = train_df.iloc[10]['highlights']

In [77]:
high_2

'The distortion is the result of the Mercator map which was created in 1596 to help sailors navigate the world .\nIt gives the right shapes of countries but at the cost of distorting sizes in favour of the wealthy lands to the north .\nFor instance, north America looks larger, or at least as big, as Africa, and Greenland also looks of comparable size .\nIn reality, you can fit north America into Africa and still have space for India, Argentina, Tunisia and some left over .\nMap suggests Scandinavian countries are larger than India, whereas in reality India is three times the size .\nThe biggest challenge for cartographers is that it is impossible to portray reality of spherical world on a flat map .'

In [79]:
model.push_to_hub("badbrock/llama3-1b-lora-summary")
tokenizer.push_to_hub("badbrock/llama3-1b-lora-summary")

README.md:   0%|          | 0.00/607 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

Saved model to https://huggingface.co/badbrock/llama3-1b-lora-summary


tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/badbrock/llama3-1b-lora-summary/commit/295b7ecbea4cd2fe46c7b230b2951a8f9a154f53', commit_message='Upload tokenizer', commit_description='', oid='295b7ecbea4cd2fe46c7b230b2951a8f9a154f53', pr_url=None, repo_url=RepoUrl('https://huggingface.co/badbrock/llama3-1b-lora-summary', endpoint='https://huggingface.co', repo_type='model', repo_id='badbrock/llama3-1b-lora-summary'), pr_revision=None, pr_num=None)